In [1]:
# Autoreload 
%load_ext autoreload
%autoreload 2

"""Set random seed"""
import torch 
torch.manual_seed(0) 
import numpy as np
np.random.seed(0)

from pathlib import Path


,uniprot_id,mod_rsd,mod,site,res,pos
0,P31946,T2-p,p,T2,T,2
1,P31946,S6-p,p,S6,S,6
2,P31946,Y21-p,p,Y21,Y,21
3,P31946,T32-p,p,T32,T,32
4,P31946,S39-p,p,S39,S,39
...,...,...,...,...,...,...
239815,Q8IYH5,S474-p,p,S474,S,474
239816,Q8IYH5,S606-p,p,S606,S,606
239817,Q8IYH5,Y670-p,p,Y670,Y,670
239818,Q8IYH5,S677-p,p,S677,S,677


In [2]:
filepath = Path("./indexes_dict.json") 

from phosphosite.utils.io import save_index_dict, load_index_dict
indexes_dict = load_index_dict(filepath=filepath)

indexes_dict

{'Q92547': {'idx': tensor([ 141,  152,  300,  349,  369,  491,  497,  498,  553,  586,  740,  746,
           804,  816,  835,  852,  859,  863,  877,  878,  885,  887,  997, 1001,
          1011, 1015, 1092, 1125, 1137, 1158, 1182, 1191, 1215, 1503,  184,  187,
           297,  299,  617,  741,  775,  778,  821,  847,  860, 1061, 1063, 1104,
          1220, 1230,  186,  493, 1490,  957,  726,  664,  178, 1056,  454,  309,
           414,  623,  135,  262, 1386,  940,  106,   55, 1272, 1461,  364,   57,
           114,   37,  916,  986, 1348]),
  'y': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.])},
 'Q5JSP0': {'idx': tensor([ 55,  67, 100, 127, 135, 396, 413, 445, 485, 525, 546, 6

In [14]:
from create_dataset import dataset as ds
ds

PhosphositeGraphDataset(17067)

In [13]:
ds[0]

TypeError: 'NoneType' object is not subscriptable

In [8]:
for i, uid in enumerate(ds.uniprot_ids):
    print(uid, ds[i].name) 


TypeError: 'NoneType' object is not subscriptable

In [33]:
import glob
from tqdm import tqdm
from typing import List, Optional, Union, Tuple, Dict, Any
import torch_geometric

def add_labels_to_dataset(
    from_dir: Path,
    to_dir: Path,
    uniprot_ids: List[str] = None,
    extension: str = ".pt",
    indexes_dict: Dict[str, Any] = None, 
    overwrite: bool = False, 
    skip_if_exists: bool = True, 
): 
    
    # Get all `.pt` files in `from_dir`
    files = glob.glob(str(from_dir / "processed" / f"*{extension}"))
    files = [Path(f) for f in files]
    # Filter files by `uniprot_ids`
    if uniprot_ids is not None:
        files = [f for f in files if f.stem in uniprot_ids]
    
    files = sorted(files)

    # Iterate over all files
    pbar = tqdm(files)
    for filename in pbar:

        outfile = to_dir / "processed" / f"{filename.stem}{extension}"
        if outfile.exists() and not overwrite:
            continue

        # Update progress bar description
        pbar.set_description(f"{filename.stem}")
        
        data = torch.load(filename)

        if not isinstance(data, torch_geometric.data.Data):
            continue

        assert data.name == filename.stem, f"{data.name} != {filename.stem}"
        if data.name not in indexes_dict.keys():
            continue
        uniprot_id = data.name

        # check if attributes already set. 
        if hasattr(data, "y") and hasattr(data, "y_index"):
            if skip_if_exists:
                continue
        
        data.y          = indexes_dict[uniprot_id]["y"]
        data.y_index    = indexes_dict[uniprot_id]["idx"]
            
        
        torch.save(data, outfile)

In [22]:
from_dir = Path("./protein_graph_dataset/")
to_dir = Path("./labelled_protein_graph_dataset/")


# Get first 10 uniprot_ids from from_dir
uniprot_ids = [f.stem for f in (from_dir / "processed").glob("*.pt")][:10]
uniprot_ids

['P51816',
 'P00390',
 'Q8IZF7',
 'Q6ZN92',
 'Q9H6L5',
 'Q7Z7K2',
 'A6NEH6',
 'O75293',
 'Q92636',
 'Q8NHC7']

In [23]:
add_labels_to_dataset(
    from_dir=from_dir,
    to_dir=to_dir,
    uniprot_ids=uniprot_ids,
    indexes_dict=indexes_dict,
)

Q9H6L5: 100%|██████████| 10/10 [00:00<00:00, 14.25it/s]


In [25]:
# Create dataset 

from create_dataset import get_dataset

dsl = get_dataset(
    root_dir=to_dir,
    uniprot_ids=uniprot_ids,
    index_dict=None,
)

No `y_label_map` provided, assuming that `.pt` files already contain `y` and `y_index` attributes.
No `y_label_map` provided, assuming that `.pt` files already contain `y` and `y_index` attributes.


In [28]:
dsl[0].y_index

tensor([ 345,  429,  430,  719,  789,  884,  899,  900,  917,  930, 1219,  340,
         344,  396,  400,  516,  974, 1123, 1126, 1135,  354, 1193, 1124, 1290,
        1295, 1250, 1129, 1163,  198, 1051,  659,  651,  621,  184,  542,  351,
         415,  145,  508,  448,  383,  372,  154,  992,   60,   59,  287,  523,
        1067,  538,  968,  366,  369, 1054])

### Re-run for everything. 

In [35]:
add_labels_to_dataset(
    from_dir=from_dir,
    to_dir=to_dir,
    uniprot_ids=None, # use all processed filenames.
    indexes_dict=indexes_dict,
    overwrite=False, 
)

pre_transform: 100%|██████████| 17067/17067 [00:00<00:00, 88529.32it/s]


In [36]:
# Check if all labels are set
dsl = get_dataset(
    root_dir=to_dir,
    uniprot_ids=None,
    index_dict=None,
)

No `y_label_map` provided, assuming that `.pt` files already contain `y` and `y_index` attributes.


In [38]:
for d in dsl:
    assert hasattr(d, "y")
    assert hasattr(d, "y_index")

In [39]:
to_dir

PosixPath('labelled_protein_graph_dataset')